In [104]:
from google.colab import drive
drive.mount('/content/drive')

# 2) csv 파일 불러오기
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터/test.csv')

train_df = train_df.drop(columns=['ID'])
test_df = test_df.drop(columns=['ID'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 결측치 처리

### Random Forest

**배터리용량 예측에서 변수중요도 파악**

**보증기간 7년을 기준으로 나눠서 RF모델로 결측치 예측**

In [105]:
# 7년 이상/미만 데이터 분리
train_over_7 = train_df[train_df['보증기간(년)'] >= 7]
train_under_7 = train_df[train_df['보증기간(년)'] < 7]

# 각 그룹별 결측/비결측 데이터 분리
non_null_over_7 = train_over_7[train_over_7['배터리용량'].notnull()]
null_over_7 = train_over_7[train_over_7['배터리용량'].isnull()]

non_null_under_7 = train_under_7[train_under_7['배터리용량'].notnull()]
null_under_7 = train_under_7[train_under_7['배터리용량'].isnull()]

# 특성 정의
features = ['차량상태', '주행거리(km)']

# 7년 이상 모델
X_over_7 = pd.get_dummies(non_null_over_7[features])
y_over_7 = non_null_over_7['배터리용량']
model_over_7 = RandomForestRegressor(random_state=42)
model_over_7.fit(X_over_7, y_over_7)

# 7년 미만 모델
X_under_7 = pd.get_dummies(non_null_under_7[features])
y_under_7 = non_null_under_7['배터리용량']
model_under_7 = RandomForestRegressor(random_state=42)
model_under_7.fit(X_under_7, y_under_7)

# 결측치 예측을 위한 데이터 준비
# 학습 데이터의 columns와 동일하게 맞추기
X_null_over_7 = pd.get_dummies(null_over_7[features]).reindex(columns=X_over_7.columns, fill_value=0)
X_null_under_7 = pd.get_dummies(null_under_7[features]).reindex(columns=X_under_7.columns, fill_value=0)

# 결측치 예측
null_over_7['배터리용량'] = model_over_7.predict(X_null_over_7)
null_under_7['배터리용량'] = model_under_7.predict(X_null_under_7)

# 원래 train_df에 업데이트
train_df.update(null_over_7)
train_df.update(null_under_7)

<ipython-input-105-4eee6854ed07>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_over_7['배터리용량'] = model_over_7.predict(X_null_over_7)
<ipython-input-105-4eee6854ed07>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_under_7['배터리용량'] = model_under_7.predict(X_null_under_7)


In [106]:
# 테스트 데이터를 7년 기준으로 분리
test_over_7 = test_df[test_df['보증기간(년)'] >= 7]
test_under_7 = test_df[test_df['보증기간(년)'] < 7]

# 각 그룹별 결측치 데이터 분리
# == -1 대신 isnull() 사용
null_test_over_7 = test_over_7[test_over_7['배터리용량'].isnull()]
null_test_under_7 = test_under_7[test_under_7['배터리용량'].isnull()]

# 특성 정의
features = ['차량상태', '주행거리(km)']

# 예측을 위한 데이터 준비
X_null_over_7 = pd.get_dummies(null_test_over_7[features]).reindex(columns=X_over_7.columns, fill_value=0)
X_null_under_7 = pd.get_dummies(null_test_under_7[features]).reindex(columns=X_under_7.columns, fill_value=0)

# 결측치 예측
null_test_over_7['배터리용량'] = model_over_7.predict(X_null_over_7)
null_test_under_7['배터리용량'] = model_under_7.predict(X_null_under_7)

# 원본 데이터프레임 업데이트
test_df.update(null_test_over_7)
test_df.update(null_test_under_7)

<ipython-input-106-ec956d39e8e8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_test_over_7['배터리용량'] = model_over_7.predict(X_null_over_7)
<ipython-input-106-ec956d39e8e8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_test_under_7['배터리용량'] = model_under_7.predict(X_null_under_7)


# RF로 결측치 예측

In [59]:
from sklearn.ensemble import RandomForestRegressor

# 결측이 아닌 데이터를 기반으로 모델 학습
non_null_data = train_df[train_df['배터리용량'].notnull()]
null_data = train_df[train_df['배터리용량'].isnull()]

features = [ '차량상태', '주행거리(km)']  # 유의미한 변수 선택
X_train = pd.get_dummies(non_null_data[features], drop_first=True)
y_train = non_null_data['배터리용량']

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 결측치 예측
X_null = pd.get_dummies(null_data[features], drop_first=True)
null_data['배터리용량'] = model.predict(X_null)
train_df.update(null_data)

<ipython-input-59-44fb7da01e98>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_data['배터리용량'] = model.predict(X_null)


In [60]:
### 테스트 데이터에도 동일하게 진행
# 결측이 아닌 데이터를 기반으로 모델 학습
non_null_data = test_df[test_df['배터리용량'].notnull()]
null_data = test_df[test_df['배터리용량'].isnull()]

features = [ '차량상태', '주행거리(km)']  # 유의미한 변수 선택
X_train = pd.get_dummies(non_null_data[features], drop_first=True)
y_train = non_null_data['배터리용량']

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 결측치 예측
X_null = pd.get_dummies(null_data[features], drop_first=True)
null_data['배터리용량'] = model.predict(X_null)
test_df.update(null_data)

<ipython-input-60-0af7754854fa>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_data['배터리용량'] = model.predict(X_null)


# Feature Engineerng

### 변수관련 변수 생성

In [107]:
import numpy as np

# 1. 차량 나이 관련 피처
reference_year = 2025
train_df['차량나이'] = reference_year - train_df['연식(년)']
test_df['차량나이'] = reference_year - test_df['연식(년)']

train_df['차량나이'] = train_df['차량나이'].apply(lambda x: x if x > 0 else 0)
test_df['차량나이'] = test_df['차량나이'].apply(lambda x: x if x > 0 else 0)

train_df['보증잔여기간'] = train_df['보증기간(년)'] - train_df['차량나이']
test_df['보증잔여기간'] = test_df['보증기간(년)'] - test_df['차량나이']

# 2. 주행거리 관련 피처
train_df['연간_주행거리'] = train_df['주행거리(km)'] / train_df['차량나이'].replace(0, np.nan)
test_df['연간_주행거리'] = test_df['주행거리(km)'] / test_df['차량나이'].replace(0, np.nan)

train_df['연간_주행거리'] = train_df['연간_주행거리'].fillna(train_df['주행거리(km)'])
test_df['연간_주행거리'] = test_df['연간_주행거리'].fillna(test_df['주행거리(km)'])

train_df['주행거리_구간'] = pd.qcut(train_df['주행거리(km)'], q=5, labels=['매우적음', '적음', '보통', '많음', '매우많음'])
test_df['주행거리_구간'] = pd.qcut(test_df['주행거리(km)'], q=5, labels=['매우적음', '적음', '보통', '많음', '매우많음'])

# 3. 배터리 용량 관련 피처
train_df['배터리용량_대비_주행거리'] = train_df['주행거리(km)'] / train_df['배터리용량'].replace(0, np.nan)
test_df['배터리용량_대비_주행거리'] = test_df['주행거리(km)'] / test_df['배터리용량'].replace(0, np.nan)

train_df['배터리용량_대비_주행거리'] = train_df['배터리용량_대비_주행거리'].fillna(train_df['주행거리(km)'])
test_df['배터리용량_대비_주행거리'] = test_df['배터리용량_대비_주행거리'].fillna(test_df['주행거리(km)'])

# 4. 제조사 및 모델 관련 피처
train_df['제조사_모델'] = train_df['제조사'] + '_' + train_df['모델']
test_df['제조사_모델'] = test_df['제조사'] + '_' + test_df['모델']

manufacturer_mean_km = train_df.groupby('제조사')['주행거리(km)'].mean()
train_df['제조사_평균_주행거리'] = train_df['제조사'].map(manufacturer_mean_km)
test_df['제조사_평균_주행거리'] = test_df['제조사'].map(manufacturer_mean_km)

# 5. 사고 및 구동방식 관련 피처
accident_map = {'Yes': 1, 'No': 0}
train_df['사고여부'] = train_df['사고이력'].replace(accident_map)
test_df['사고여부'] = test_df['사고이력'].replace(accident_map)

# 6. 상태 및 텍스트 정보 활용
train_df['차량상태_분류'] = train_df['차량상태']
test_df['차량상태_분류'] = test_df['차량상태']

train_df['모델_숫자'] = train_df['모델'].str.extract(r'(\d+)')
test_df['모델_숫자'] = test_df['모델'].str.extract(r'(\d+)')


<ipython-input-107-ccbda4e77755>:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['사고여부'] = train_df['사고이력'].replace(accident_map)
<ipython-input-107-ccbda4e77755>:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['사고여부'] = test_df['사고이력'].replace(accident_map)


### 카테고리 결합 변수 생성

In [108]:
# 1. 카테고리 결합
train_df['제조사_모델'] = train_df['제조사'] + '_' + train_df['모델']
test_df['제조사_모델'] = test_df['제조사'] + '_' + test_df['모델']

train_df['상태_사고여부'] = train_df['차량상태'] + '_' + train_df['사고여부'].astype(str)
test_df['상태_사고여부'] = test_df['차량상태'] + '_' + test_df['사고여부'].astype(str)

train_df['구동방식_상태'] = train_df['구동방식'] + '_' + train_df['차량상태']
test_df['구동방식_상태'] = test_df['구동방식'] + '_' + test_df['차량상태']

# 2. 범주형 변수 간 상호작용
train_df['제조사_구동방식'] = train_df['제조사'] + '_' + train_df['구동방식']
test_df['제조사_구동방식'] = test_df['제조사'] + '_' + test_df['구동방식']

train_df['구동방식_사고여부'] = train_df['구동방식'] + '_' + train_df['사고여부'].astype(str)
test_df['구동방식_사고여부'] = test_df['구동방식'] + '_' + test_df['사고여부'].astype(str)

train_df['상태_연간주행거리'] = train_df['차량상태'] + '_' + pd.qcut(train_df['연간_주행거리'], 5, labels=False).astype(str)
test_df['상태_연간주행거리'] = test_df['차량상태'] + '_' + pd.qcut(test_df['연간_주행거리'], 5, labels=False).astype(str)


In [109]:
# 차량 상태별 평균 주행거리 계산
avg_mileage_by_condition = train_df.groupby('차량상태')['주행거리(km)'].mean()

# 평균 주행거리 변수 추가
train_df['차량상태_평균_주행거리'] = train_df['차량상태'].map(avg_mileage_by_condition)
test_df['차량상태_평균_주행거리'] = test_df['차량상태'].map(avg_mileage_by_condition)

# 차량 상태별 평균 보증잔여기간 계산
avg_warranty_by_condition = train_df.groupby('차량상태')['보증잔여기간'].mean()

# 평균 보증잔여기간 변수 추가
train_df['차량상태_평균_보증잔여기간'] = train_df['차량상태'].map(avg_warranty_by_condition)
test_df['차량상태_평균_보증잔여기간'] = test_df['차량상태'].map(avg_warranty_by_condition)

# 제조사, 구동방식, 차량상태 조합 변수 생성
train_df['제조사_구동방식_차량상태'] = train_df['제조사'] + '_' + train_df['구동방식'] + '_' + train_df['차량상태']
test_df['제조사_구동방식_차량상태'] = test_df['제조사'] + '_' + test_df['구동방식'] + '_' + test_df['차량상태']

# 라벨 인코딩

In [110]:
from sklearn.preprocessing import LabelEncoder

# object 타입 변수 선택
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns
categorical_cols = test_df.select_dtypes(include=['object', 'category']).columns

# 모든 데이터셋에 대해 동일한 LabelEncoder 적용
label_encoders = {}

for col in categorical_cols:
    # LabelEncoder 생성
    le = LabelEncoder()

    # train과 test의 unique 값들을 모두 포함하여 fit
    unique_values = pd.concat([
        train_df[col],
        test_df[col]
    ]).unique()

    # LabelEncoder 학습
    le.fit(unique_values)

    # 각 데이터셋에 transform 적용
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

    # 나중에 사용할 수 있도록 label_encoders 저장
    label_encoders[col] = le

# 저장

In [111]:
# 저장 경로 설정
train_output_path = '/content/drive/MyDrive/Colab Notebooks/데이터/train_processed2.csv'
test_output_path = '/content/drive/MyDrive/Colab Notebooks/데이터/test_processed2.csv'

# 데이터 저장
train_df.to_csv(train_output_path, index=False)
test_df.to_csv(test_output_path, index=False)

# 변수 중요도

In [112]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 데이터 분할
X = train_df.drop('가격(백만원)', axis=1)
y = train_df['가격(백만원)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost 모델 생성 및 학습
model = RandomForestRegressor(
    max_depth=8,
    random_state=42
)

# 모델 학습
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=8, random_state=42)

In [113]:
# 변수 중요도 확인
feature_importance_df = pd.DataFrame({
    'Feature' : X.columns,
    'Importance' : model.feature_importances_
}).sort_values(by = 'Importance', ascending=False)

print('변수중요도:')
print(feature_importance_df)

변수중요도:
           Feature    Importance
15     제조사_평균_주행거리  6.024263e-01
26   제조사_구동방식_차량상태  1.310035e-01
1               모델  8.456495e-02
18           모델_숫자  7.561599e-02
14          제조사_모델  5.769506e-02
21        제조사_구동방식  2.207304e-02
0              제조사  1.473503e-02
6          보증기간(년)  2.476015e-03
10          보증잔여기간  1.698682e-03
20         구동방식_상태  1.266462e-03
3            배터리용량  1.232387e-03
5         주행거리(km)  7.669132e-04
13   배터리용량_대비_주행거리  7.032748e-04
25  차량상태_평균_보증잔여기간  7.031007e-04
11         연간_주행거리  6.782455e-04
17         차량상태_분류  4.489155e-04
23       상태_연간주행거리  3.509160e-04
22       구동방식_사고여부  3.315456e-04
24    차량상태_평균_주행거리  3.187682e-04
2             차량상태  2.352628e-04
4             구동방식  2.300342e-04
19         상태_사고여부  2.210729e-04
8            연식(년)  1.175176e-04
9             차량나이  9.462494e-05
12         주행거리_구간  1.134791e-05
16            사고여부  7.055588e-07
7             사고이력  3.216576e-07


In [114]:
train_df.isnull().sum()

,0
제조사,0
모델,0
차량상태,0
배터리용량,0
구동방식,0
주행거리(km),0
보증기간(년),0
사고이력,0
연식(년),0
가격(백만원),0


In [115]:
test_df.isnull().sum()

,0
제조사,0
모델,0
차량상태,0
배터리용량,0
구동방식,0
주행거리(km),0
보증기간(년),0
사고이력,0
연식(년),0
차량나이,0
